In [1]:
# this is to supress an internal warning that was being raised by category_encoders
# you can safely ignore this
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import category_encoders
import json
import pandas as pd
import pickle
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

# Preserving your model

In this learning unit you will learn how to preserve your model so that
the value it generates can be used in a separate process or program than the
one in which it was fitted. There are a few different ways to do this
but for this specialization we will be using a few tools:

1. [pickle](https://docs.python.org/3/library/pickle.html) from python core
1. [pipelines](https://scikit-learn.org/stable/modules/compose.html#pipelines-and-composite-estimators) from scikit

## The journey

### Train
We are going to first train a model on the classic titanic dataset. We will use
this one because it has categorical and numeric features, and missing values in both types.

### Serialize
Once the model has been trained as part of a pipeline, we will [serialize](https://en.wikipedia.org/wiki/Serialization)
it using the [pickle](https://docs.python.org/3/library/pickle.html) package
that is found in python's core.

### Predict on new data
After we are confident we can retrieve the pickled model from disk, we will
show how to prepare a brand new observation for prediction with the model.

Let's get started! We're not going to spend much time preparing the dataset or working on model performance
because it's not the focus of this learning unit. So let's power through the first few steps!

In [3]:
# read original dataset from disk and take a look at it
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# let's get rid of a few features that don't hold anything
# particularly useful and take another peak
df = df.drop(['Ticket', 'Name', 'PassengerId'], axis=1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [5]:
# now let's split it into X_train and y_train
X_train, y_train = df.drop('Survived', axis=1), df.Survived

## Let's build the pipeline

Okay, the next bit of necessary code isn't very much at all but
is very dense. So let's take things one at a time to understand
the motivation.

We'll begin with just the model itself - a logistic regression
and see what happens.

In [6]:
# NOTE: We're using a try/except block here because (SPOILER ALERT)
# the fit function is going to fail and we want the notebook to look clean.
# Go ahead and remove the try/except block to see the error's stack trace

try:
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
except ValueError as e:
     print(e)

could not convert string to float: 'male'


We know this game - scikit classifiers don't know how to deal
with non-numerical data. Since we already know about pipelines,
let's try to put together a pipeline that has a OneHotEncoder
in an attempt to deal with the non-numeric data.

In [7]:
try:
    pipeline = make_pipeline(
        OneHotEncoder(
            categories=list(
                X_train.select_dtypes(include=['object']).columns)),
        LogisticRegression()
    )
    pipeline.fit(X_train, y_train)
except ValueError as e:
     print(e)

Input contains NaN, infinity or a value too large for dtype('float64').


The [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
from scikit can't deal with categories with missing values. We could replace these missing values with a new category (like `unknown`), but it's easier to use the [category_encoders](http://contrib.scikit-learn.org/category_encoders/index.html) package, that deals with this out of the box.

We'll use the [OneHotEncoder](http://contrib.scikit-learn.org/category_encoders/onehot.html). This one is a bit smarter than scikit's one, as the default behavior is for all string columns to be dummified.

One last thing to note is the usage of the `handle_unknown` keyword, which is set to `value`. This is the default behaviour, but it's worth understanding what is happening. Later on, at prediction time, if the encoder runs into a value that it hasn't seen before, it won't throw an error. Instead, all the dummy features that were created will have value `0` for this observation.
It's not a perfect solution to be silently ignoring stuff that you haven't seen before but hey, it's better than crashing!

In [8]:
try:
    pipeline = make_pipeline(
        category_encoders.OneHotEncoder(handle_unknown='value'),
        LogisticRegression(),
    )
    pipeline.fit(X_train, y_train)
except ValueError as e:
     print(e)

Input contains NaN, infinity or a value too large for dtype('float64').


Ups, we're still having the same error... 
We've dealt with NaNs in the categorical columns, but some of the numeric columns still have NaNs in them. Now we need to use the [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)
for those bad boys. SimpleImputer fills in missing values in the data. By default, it assumes that the missing values are NaNs, but we could specify other types of missing values with the `missing_values` keyword. With the `strategy` keyword we define how to fill in the missing values: in this case, we're replacing them with the mean value of the feature where the missing value occurs.

In [9]:
pipeline = make_pipeline(
    category_encoders.OneHotEncoder(),
    SimpleImputer(strategy='mean'),
    LogisticRegression(max_iter=1000),
)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Sex', 'Cabin', 'Embarked'])),
                ('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

Hooray! Now we are using a pipeline and training a classifier without doing
any explicit preprocessing of the dataset!

With the pipeline we should now be able to move on with our lives and start
processing new observations.

## Predicting on new observations

Let's construct a new observation using a protocol that is technology agnostic: [json](https://en.wikipedia.org/wiki/JSON).

We will assume that a new observation has come over the wire using a transportation
layer such as HTTP, which means that it will arrive to us as a json string (a string whose content follows the json protocol).

In [10]:
new_obs_str = '{"Age": 22.0, "Cabin": null, "Embarked": "S", "Fare": 7.25, "Parch": 0, "Pclass": 3, "Sex": "male", "SibSp": 1}'

Great, now we've got a new observation as a json string. This is desireable because no matter what
programming language or environment we are in, we know that there will be support for deserialization
into a native type. In ruby these are hashes, in javascript they are objects, and in python they are
dictionaries.

So let's turn our json string into a dictionary - it's a great starting point to do anything we may need.

In [11]:
new_obs_dict = json.loads(new_obs_str)
print('type {}'.format(type(new_obs_dict)))

type <class 'dict'>


In [12]:
new_obs_dict

{'Age': 22.0,
 'Cabin': None,
 'Embarked': 'S',
 'Fare': 7.25,
 'Parch': 0,
 'Pclass': 3,
 'Sex': 'male',
 'SibSp': 1}

Not so fast... scikit models don't know how to deal with dictionaries!

Well, we know that when we trained the model, the pipeline took a pandas
dataframe so that's what we should be passing into `predict_proba` as well.

With that in mind, let's take a few lines of code to transform the dictionary
into a pandas dataframe. Note that a series isn't good enough, it must be
a full dataframe, even if it's just for a single observation.

Although it's only a few lines of code, it's pretty dense, so be sure to
read the comments in order to fully understand what's going on here.

In [13]:
# First step is to create a dataframe with the columns in the correct
# order. You can get the correct order by getting the columns from
# the X_train dataframe with which the model was trained. Doing this
# will preserve the correct order.

# Also note that you must pass the dictionary as an entry
# in an array, even if there is only a single one... scikit models
# always assume things are being processed in batches.
obs = pd.DataFrame([new_obs_dict], columns=X_train.columns.tolist())

# Now you need to make sure that the types are correct so that the
# pipeline steps will have things as expected.
obs = obs.astype(X_train.dtypes)

In [14]:
obs

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.25,None,S


In [15]:
# Finally you can call predict_proba and the pipeline will function
# as expected.
pipeline.predict_proba(obs)

array([[0.83902602, 0.16097398]])

Alright, we are feeling pretty cool right about now. We have a trained
model, we can process new observations! Now that we have this under control
let's take a look at preserving our model so that we can keep
this sweetness for posterity.

## Serialization of the necessary components

Okay let's take a moment to imagine that we will need to use
this model in a totally different environment than the one it was trained in.

For instance, if we want to share the model with a friend: we woulnd't want them to retrain it (some models take hours to train), and we wounldn't want to have to send them the training data (it might be very large and it might be confidential data that is not suitable to be shared).

Even withing the same computer, we may want to use the model in a different notebook than the one it was trained in. Or in a different python process. Or in a flask server (hint hint).

What we  want to do is to save the model to the disk so that it can be transfered somewhere else and used later on.

Remember that serialization is just the process of storing something
so that it can be deserialized and used later on. Let's think
about what we need in order to be able to call
`predict_proba` on a new observation. It is:

1. The column names in the correct order
1. The fitted pipeline
1. The dtypes of the columns of the training set

One at a time, let's look at serializing these.

### Serializing the columns in the correct order

Probably the most well-known serialization format for data is json.

This is great because it's robust and technology agnostic. Let's serialize the
columns of the training set in the correct order:

In [16]:
with open('columns.json', 'w') as fh:
    json.dump(X_train.columns.tolist(), fh)

### Serializing the fitted pipeline

Now we need to serialize the fitted pipeline. Unfortunately we
don't have something as clean as json to do this with since 
the pipeline is a python object and not just raw data (like the 
column names are).

In order to preserve the pipeline, we will need to use a library to export a python object (the fitted pipeline) into our hard drive in a way that python can reload it again. The most common library for serialization of python objects is [pickle](https://docs.python.org/3/library/pickle.html), which is part of the core python language (the standard library). However scikit-learn comes with a version of pickle designed to save scikit-learn estimators, [joblib](http://scikit-learn.org/stable/modules/model_persistence.html).

In [17]:
import joblib
joblib.dump(pipeline, 'pipeline.pickle') 

['pipeline.pickle']

**NOTE**. One thing to consider when serializing/deserializing scikit-learn estimators is that joblib/pickle don't store the definitions of the estimators (the code). This means 2 things:

- All of the libraries you use to build the pipeline on your laptop need to be available (installed) in the machine where you deploy it. 

- All of the custom code you use (for example, any custom transformer you build) in the pipeline needs to be defined as well when you load the pipeline back.

### Serializing the dtypes of the columns

We have a similar situation with the dtypes. When you call `X_train.dtypes`, you will get a list of python
objects, so we must use pickle to serialize them as well.

In [18]:
with open('dtypes.pickle', 'wb') as fh:
    pickle.dump(X_train.dtypes, fh)

Alrighty then, we have now serialized all the necessary components
of our model in order to be able to process new observations!

Move on to the next notebook to see how we can deserialize and use
all of this work in a completely different process.